<a href="https://colab.research.google.com/github/robimalco/futbin_scraper/blob/master/futbin_scraper_prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [0]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import datetime

# Extract HTML of the page and convert it to soup

In [0]:
url = 'https://www.futbin.com/20/player/846/neymar-jr'
response_url = requests.get(url)

In [0]:
soup_response_url = BeautifulSoup(response_url.text)

# Extract main KPIs for a player

In [0]:
core_kpi_section = soup_response_url.find('div', {'id': 'player_stats_json'}).getText()
json_kpi_section = json.loads(core_kpi_section)
player_type = 1

df_player_kpis = pd.DataFrame(columns=['player_type', 'category', 'kpi_name', 'kpi_value'])

for player in json_kpi_section:
  for category in player:
    for variable in player[category]:
      kpi_name = variable['name']
      kpi_value = variable['value']
      df_player_kpis = df_player_kpis.append({
          'player_type': player_type,
          'category': category,
          'kpi_name': kpi_name,
          'kpi_value': kpi_value}, ignore_index=True)
  player_type = player_type + 1

In [0]:
df_player_kpis.to_csv('output_data.csv')

# Extract Hourly player value

In [0]:
# https://www.futbin.com/20/playerGraph?type=today&year=20&player=190871&set_id=

In [0]:
player_ids = soup_response_url.find_all('div', {'class': 'pversion'})[1].find('a')['data-resource'].replace('p', '')
url_hourly = 'https://www.futbin.com/20/playerGraph?type=today&year=20&player=' + str(player_ids)

In [0]:
response_url_hourly = requests.get(url_hourly)
soup_response_url_hourly = BeautifulSoup(response_url_hourly.text)

In [0]:
json_response_url_hourly = json.loads(soup_response_url_hourly.getText())

In [0]:
df_player_hourly = pd.DataFrame(columns=['console', 'time', 'value'])
for console in json_response_url_hourly:
  for data_point in json_response_url_hourly[console]:
    df_player_hourly = df_player_hourly.append({
        'console': console,
        'time': datetime.datetime.fromtimestamp(data_point[0] / 1000).strftime('%c'),
        'value': data_point[1]}, ignore_index=True)
df_player_hourly